# OpenAI API 기초 실습

## 1. openai python sdk 설치
- openai api는 파이썬용 SDK 를 설치하거나 HTTP Client 를 사용하여 REST API 를 직접 호출하는 방식으로 사용할 수 있습니다.
- 프로그래밍 언어 선택에 제약이 있는 경우를 제외하면 대부분 파이썬 SDK 를 설치하여 사용합니다.

In [1]:
!pip install openai

## 2. Azure OpenAI 리소스 정보 설정
- openai API 를 호출하기 위해서는 API key 를 입력해야합니다.
- Azure OpenAI 의 경우, API key 뿐만 아니라 endpoint, deployment name 등이 추가로 필요합니다.
- 이 실습 환경에서는 Azure OpenAI 를 사용하기 위해 필요한 정보를 환경변수로 제공합니다.

In [3]:
#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

## 3. 간단한 Chat Completion 구현
- AzureOpenAI 클라이언트를 생성하고 chat completions create 를 호출하여 간단히 GPT 의 응답을 받을 수 있습니다.

In [4]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = AOAI_ENDPOINT, 
  api_key=AOAI_API_KEY,  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model=AOAI_DEPLOY_GPT4O_MINI,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕?"}
    ]
)

print(response.choices[0].message.content)    # LLM 답변
print(response.usage)                         # 토큰 사용량
print(response.prompt_filter_results)         # 프롬프트 필터 결과

안녕하세요! 어떻게 도와드릴까요?
CompletionUsage(completion_tokens=10, prompt_tokens=20, total_tokens=30, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]


## 4. Multi-Modal (GPT-Vision) 구현
- o1, GPT-4o, GPT-4o-mini 및 GPT-4 Turbo with Vision 모델은 이미지를 분석하여 자연어로 답변할 수 있습니다.

In [5]:
#이미지 확인
from IPython.display import display, Image

image_url = "https://img.animalplanet.co.kr/news/2019/11/28/700/f9in35p5660ce423x290.jpg"
display(Image(url=image_url, width=400))

In [7]:
response = client.chat.completions.create(
    model=AOAI_DEPLOY_GPT4O,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {
                "type": "text",
                "text": "이미지를 설명해줘"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": image_url
                }
            }]
        }
    ]
)

print(response.choices[0].message.content)

이미지에는 흰색 털을 가진 귀여운 동물의 귀 부분이 보입니다. 배경은 흐릿하며 실내 또는 외부의 어두운 환경으로 추정됩니다.


## 5. 로컬 파일을 활용한 이미지 분석
- Base64 이미지로 변환하면 로컬 이미지도 분석할 수 있습니다.
- 공식 문서를 참고하면 쉽게 구현할 수 있습니다. (https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/gpt-with-vision?tabs=rest)

In [10]:
os.listdir('.')

['.ipynb_checkpoints',
 'langchain_toolcalling.ipynb',
 'test.png',
 'azure_openai_first.ipynb',
 'langchain_multimodal.ipynb']

In [ ]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Example usage
image_path = 'test.png'
data_url = local_image_to_data_url(image_path)
print("Data URL:", data_url[:100])

Data URL: data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAA5wAAALfCAIAAACCa1rwAAAAAXNSR0IArs4c6QAAAHhlWElmTU0AKg


In [9]:
response = client.chat.completions.create(
    model=AOAI_DEPLOY_GPT4O_MINI,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {
                "type": "text",
                "text": "이미지를 설명해줘"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }]
        }
    ]
)

print(response.choices[0].message.content)

이 이미지는 마이크로서비스 아키텍처의 한 예를 보여줍니다. 다음과 같은 구성 요소들이 포함되어 있습니다:

1. **Service 1, Service 2, Service 3**: 이들은 각각 독립적으로 운영되는 마이크로서비스입니다. 각 서비스는 특정 기능을 수행하며, 서로 협력하여 전체 시스템을 구성합니다.

2. **HTTP Gateway**: 이 구성 요소는 클라이언트 요청을 받고 이를 적절한 서비스로 라우팅하는 역할을 합니다. 즉, 클라이언트는 HTTP Gateway에 요청을 보내고, Gateway가 해당 요청을 적절한 서비스로 전달합니다.

3. **Database 1, Database 2, Database 3**: 각각의 서비스는 독립적인 데이터베이스와 연결되어 있습니다. 이러한 데이터베이스는 각 서비스의 데이터를 저장하고 관리합니다.

4. **화살표**: 화살표는 서비스와 데이터베이스 간의 관계를 나타냅니다. 서비스가 특정 데이터베이스에 접근하거나 데이터를 주고받는 방식을 시각적으로 표현합니다.

전체적으로, 이 그림은 마이크로서비스들이 HTTP Gateway를 통해 서로 소통하고, 각 서비스가 독자적으로 데이터베이스에 접근하는 구조를 나타내고 있습니다. 이러한 아키텍처는 시스템의 확장성과 유지보수성을 높이기 위해 자주 사용됩니다.


## 6. 대화 이력
- 문맥을 유지하며 대화하기 위해서는 대화 이력이 관리되어야합니다.
- messages에 프롬프트들을 추가하여 LLM 호출 시 대화 이력을 주입할 수 있습니다.
- "system" : 시스템 프롬프트 (o1, o3 의 경우 "developer" 로 변경되었습니다.)
- "user" : 사용자 메시지
- "assistant": LLM 응답

In [11]:
response = client.chat.completions.create(
    model=AOAI_DEPLOY_GPT4O_MINI,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "대한민국의 수도는 어디야?"},
        {"role": "assistant", "content": "대한민국의 수도는 서울입니다."},
        {"role": "user", "content": "그곳의 여행지를 추천해줄래?"}
    ]
)
print(response.choices[0].message.content)

물론입니다! 서울에는 멋진 여행지가 많이 있습니다. 다음은 서울의 추천 여행지입니다:

1. **경복궁**: 조선 왕조의 주요 궁궐 중 하나로, 아름다운 건축물과 정원을 감상할 수 있습니다. 근처의 국립민속박물관도 함께 방문해보세요.

2. **남산서울타워**: 서울의 랜드마크 중 하나로, 전망대에서 서울의 전경을 감상할 수 있습니다. 케이블카로 올라가는 재미도 있습니다.

3. **명동**: 쇼핑과 먹거리가 풍부한 유흥가로, 다양한 상점들과 맛있는 길거리 음식을 즐길 수 있습니다.

4. **인사동**: 전통 문화와 현대 미술이 어우러진 거리로, 기념품 가게와 전통 찻집이 많아 다양한 경험을 할 수 있습니다.

5. **홍대**: 젊은이들의 문화가 흐르는 지역으로, 예술적인 거리와 다양한 음식점, 카페들이 있어 활기찬 분위기를 즐길 수 있습니다.

6. **한강공원**: 한강을 따라 조성된 공원으로, 자전거 타기, 산책, 피크닉 등을 즐길 수 있는 좋은 장소입니다.

7. **북촌 한옥마을**: 전통 한옥들이 모여있는 지역으로, 한국의 전통 건축을 감상할 수 있으며, 예쁜 골목길이 인상적입니다.

8. **동대문 디자인 플라자(DDP)**: 현대적인 건축물로, 전시회와 이벤트가 자주 열리며, 주변의 동대문 시장도 인기입니다.

이 외에도 서울에는 다양한 박물관, 갤러리, 전통시장 등이 있어 즐길 거리가 많습니다. 방문할 때마다 새로운 매력을 발견할 수 있을 거예요!
